In [ ]:
import numpy as np
import scipy.sparse as sp

from src.utils import RecommenderModel, open_data, get_target_users_test, make_submission

## Opening the data

In [2]:
urm, icm = open_data()
urm, icm

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1764607 stored elements and shape (35736, 38121)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 2940040 stored elements and shape (38121, 94331)>)

In [3]:
target_users = get_target_users_test()
target_users

array([    0,     1,     2, ..., 35731, 35734, 35735])

## Building the Top Recommender model

In [4]:
class TopPopRecommender(RecommenderModel):
	def __init__(self):
		super(TopPopRecommender, self).__init__()
		self.items_popularity_ranking: np.ndarray | None = None

	def fit(self, urm: sp.csr_matrix) -> None:
		items_popularity = np.ediff1d(urm.indptr)
		self.items_popularity_ranking = np.argsort(-items_popularity)

	def recommend(self, user_id: int, at: int = 10) -> np.ndarray:
		return self.items_popularity_ranking[:at]

In [5]:
top_pop = TopPopRecommender()
top_pop.fit(urm)

## Testing the model

In [6]:
make_submission(top_pop, "top_popular.csv")